In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from IPython.display import display, Image

def crawl_imdb_movies(url):
    page_num = 0
    data = []

    while True:
        response = requests.get(f"{url}&start={page_num * 100}")
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            movie_containers = soup.find_all('div', class_='lister-item mode-advanced')

            if not movie_containers:
                break

            for container in movie_containers:
                title = container.h3.a.text

                # Tách năm phát hành từ tiêu đề
                year_text = container.h3.find('span', class_='lister-item-year').text
                year = year_text.strip('()I ')  # Loại bỏ các ký tự không cần thiết

                # Lấy thể loại
                genres = [genre.strip() for genre in container.find('span', class_='genre').text.split(',')]

                rating = float(container.strong.text)
                description = container.find_all('p', class_='text-muted')[1].text.strip()
                img_url = container.find('img')['loadlate']

                print(f"Tiêu đề: {title}")
                print(f"Năm phát hành: {year}")
                print(f"Thể loại: {', '.join(genres)}")
                print(f"Mô tả: {description}")
                print(f"Xếp hạng: {rating}")
                display(Image(url=img_url))

                data.append([title, year, ', '.join(genres), rating])


            page_num += 1
        else:
            print("Yêu cầu không thành công. Mã lỗi:", response.status_code)
            break

    return data

def save_to_csv(data):
    with open('movies.csv', mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Year', 'Genres', 'Rating'])

        for row in data:
            writer.writerow(row)

url = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&ref_=adv_nxt"

data = crawl_imdb_movies(url)
save_to_csv(data)


KeyboardInterrupt: 